In [ ]:
# If running from Colab, please uncomment and run the following cell to get summer installed.
#!pip install summerepi2==1.3.5

In [ ]:
import pandas as pd
from datetime import datetime
from summer2 import CompartmentalModel
from summer2.parameters import Parameter
pd.options.plotting.backend = 'plotly'

## Model construction and execution
The following cells constitute the epidemiological analysis of this notebook.
The code is intended to be very simple and adaptable.
Please feel free to adjust any of the data sources or parameters,
which is the purpose of this notebook.
### Country selection
First choose your country using it's ISO code,
with options only available for Malaysia, the Philippines and Vietnam.
This will determine the Our World in Data (OWID) case data 
that can be used to compare your model results against.
You can use our approximate estimate of the total population of the country,
or replace the value as per your preference.
### Dates
Feel free to edit the datetime objects that specify 
the start and end dates for the analysis, the reference ("zero") date
and the left limit for the x-axis in the final plot.
### Model
The model is a parameterised summer2 SEIR model 
with frequency-dependent transmission and 
partial observation of incidence (termed notifications).
### Results
The SEIR model provides a poor-to-moderate fit to the target data 
for the Omicron waves in each of the three countries.
Please adjust parameters and dates to achieve a better fit for your country.

In [ ]:
iso = 'MYS'

In [ ]:
cases_data = pd.read_csv('https://github.com/monash-emu/wpro_working/raw/main/data/new_cases.csv', index_col=0)[iso]
cases_data.index = pd.to_datetime(cases_data.index)
approx_pops = {
    'MYS': 33e6,
    'PHL': 114e6,
    'VNM': 97e6,
}

In [ ]:
analysis_start_date = datetime(2021, 8, 1) 
analysis_end_date = datetime(2022, 6, 1)
epi_model = CompartmentalModel(
    [analysis_start_date, analysis_end_date],
    ['susceptible', 'exposed', 'infectious', 'recovered'],
    ['infectious'],
    ref_date=datetime(2019, 12, 31),
)
epi_model.add_infection_frequency_flow('infection', Parameter('contact_rate'), 'susceptible', 'exposed')
epi_model.add_transition_flow('progression', 1.0 / Parameter('incubation_period'), 'exposed', 'infectious')
epi_model.add_transition_flow('recovery', 1.0 / Parameter('infectious_period'), 'infectious', 'recovered')
epi_model.set_initial_population({'susceptible': approx_pops[iso], 'infectious': 1.0})
incidence = epi_model.request_output_for_flow('incidence', 'progression', save_results=False)
epi_model.request_function_output('notifications', incidence * Parameter('detection_prop'));

In [ ]:
parameters = {
    'contact_rate': 0.4,
    'incubation_period': 5.0,
    'infectious_period': 5.0,
    'detection_prop': 0.07,
}
epi_model.run(parameters)

In [ ]:
plot_start_date = datetime(2021, 10, 1)
comparison_df = pd.concat([epi_model.get_derived_outputs_df(), cases_data])
comparison_df.columns = ['modelled', 'reported']
comparison_plot = comparison_df.plot()
comparison_plot.update_xaxes(range=(plot_start_date, analysis_end_date))